In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
import warnings
import pandas as pd
import imageio
from utility import get_weights, get_neurons, get_edges
%matplotlib inline

## Data Generation and Initial Fit

In [4]:
# Generating data
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1, n_samples = 500)
rng = np.random.RandomState(2)
X += 3*rng.uniform(size=X.shape)
# Scaling values before split for simplicity
X = StandardScaler().fit_transform(X)
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)
# Initialising the classifier
clf = MLPClassifier(hidden_layer_sizes=(5,10,5),alpha=1, max_iter=1000)
clf.partial_fit(X_train, y_train, classes=np.unique(y_train))

MLPClassifier(alpha=1, hidden_layer_sizes=(5, 10, 5), max_iter=1000)

## Plotting Elements

### Grid

In [8]:
# Meshgrid height
h = 0.2
# Creating a grid
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

### Neural Network graph (edges and vertices)

In [4]:
# Generating NN structuer using the model's weight matrix
neurons_df, coordinate_dict = get_neurons(clf.coefs_)
edges = get_edges(clf.coefs_)

## Define the plotting functions

In [5]:
def plot_nn(axis, neurons_df, coordinate_dict, edges):
    '''Function plotting the neural network's inner structure
    Args:
        axis (Axes object): Axis used to plot the nn's inner structure
        neurons_df (DataFrame object): DataFrame containing the (x,y) position of each neuron
        coordinate_dict (dict): Dictionary composed of (layer,neuron):{'x':pos, 'y':pos} pairs, used for fast lookups
        edges (DataFrame object): Dataframe containing the weight of each connection
    '''
    cm = plt.cm.RdBu
    c_dict = {1: '#4473a4', -1: '#d9776d', 0:'grey'}
    # Plotting weights
    for idx, row in edges.iterrows():
        parent = (row['parent_layer'],row['parent_neuron'])
        child = (row['child_layer'],row['child_neuron'])
        # Conditional formatting of high weight changes
        if np.abs(row['delta'])>0.002:
            axis.plot(
            [coordinate_dict[parent]['x'], coordinate_dict[child]['x']],
            [coordinate_dict[parent]['y'], coordinate_dict[child]['y']],
            c='red', alpha = 0.4, lw = (np.abs(row['weight'])+1)**3.5, ls = '-.', label = 'Learning'
        )
        # Regular formatting
        else:
            axis.plot(
            [coordinate_dict[parent]['x'], coordinate_dict[child]['x']],
            [coordinate_dict[parent]['y'], coordinate_dict[child]['y']],
            c='grey', alpha = 0.3, lw = (np.abs(row['weight'])+1)**3.5
        )
    # Plotting the neurons as a scatter plot
    axis.scatter(neurons_df['x'], neurons_df['y'], s=400, alpha = 1, zorder=3)

In [6]:
def plot_decision(axis, clf, X_train, y_train, X_test, y_test, xx, yy):
    '''Function plotting the data points and predicted probabilities in the feature space
    Args:
        axis (Axes object): Axis used to plot the nn's inner structure
        X_train (array): Array of training features
        y_train (array): Array of training labels
        X_test (array): Array of test features
        y_test (array): Array of test labels
        xx (array): Grid x coordinates
        yy (array): Grid y coordinates        
    '''
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    # Generating predictions
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    # Plot the contour plot
    axis.contourf(xx, yy, Z, cmap=cm, alpha=.8)
    # Plot the training points
    axis.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    # Plot the testing points
    axis.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
               edgecolors='k', alpha=0.6)
    # Removing axis labels and ticks
    axis.set_xlim(xx.min(), xx.max())
    axis.set_ylim(yy.min(), yy.max())
    axis.set_xticks(())
    axis.set_yticks(())

## Iteration

In [7]:
# List of images
files = []
# Initial weights
previous_weight = get_weights(clf.coefs_)
for i in range(300):
    # Keeps track of previous weights for comparison
    if i != 0:
        previous_weight = new_weight
    # New training iteration
    clf.partial_fit(X_train,y_train)
    # Retrieving new weights
    new_weight =  get_weights(clf.coefs_)
    # Getting the weight delta
    edges['delta'] = new_weight - previous_weight
    # Creating a figure
    fig = plt.figure(figsize=(30,10))
    # Controlling plotting context
    plot_params = {'axes.titlesize' : 16, 'axes.labelsize' : 12,
                       'axes.edgecolor': 'w', 'xtick.color': 'w', 'ytick.color':'w',
                       'ytick.major.size': 0, 'ytick.minor.size': 0 }
    with plt.rc_context(plot_params):
        ax = plt.subplot(1,2,1)
    plot_nn(ax, neurons_df, coordinate_dict, edges)
    ax2 = plt.subplot(1,2,2)
    plot_decision(ax2, clf, X_train, y_train, X_test, y_test, xx, yy)
    fig.suptitle(f'Training Epoch: {i+1}', fontsize = 32, y = 0.95)
    filename = f'video2/{i}_chart.png'
    # Saving the figure and appending filename to list
    fig.savefig(filename)
    files.append(filename)
    # Closing the figure to avoid memory problems
    plt.close()

## Movie

In [8]:
with imageio.get_writer('video2/movie.mp4', mode='I') as writer:
    # Iterate over file names
    for filename in files:
        # Read image and append to video
        image = imageio.imread(filename)
        writer.append_data(image)